This is a project I coded during an AI camp, where we learned to classify things based on text input.
Importing libraries and data:

In [1]:
# @title
import pandas as pd   # Great for tables (google spreadsheets, microsoft excel, csv).
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import spacy
import wordcloud
import os # Good for navigating your computer's files
import sys
pd.options.mode.chained_assignment = None #suppress warnings

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
nltk.download('wordnet')
nltk.download('punkt')

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python -m spacy download en_core_web_md
import en_core_web_md
text_to_nlp = spacy.load('en_core_web_md')

import scipy
# from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

def cosine(word1, word2):

  vector1 = word1.reshape(1, -1)
  vector2 = word2.reshape(1, -1)

  return cosine_similarity(vector1, vector2)[0][0]


# Import our data
!wget -q --show-progress "https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%201%20-%205/Session%203%20-%20NLP/yelp_final.csv"

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
yelp_final.csv      100%[===================>] 743.14K   797KB/s    in 0.9s    


Loading in data and removing unnecessary columns:

In [3]:
yelp_full = pd.read_csv('yelp_final.csv')
yelp_full.head()

needed_columns = ['stars', 'text']  # Replace the empty list with the column names as strings

# Using only the needed columns from the original dataset
yelp = yelp_full[needed_columns]
yelp.head()

,stars,text
0,5,My wife took me here on my birthday for breakf...
1,5,I have no idea why some people give bad review...
2,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!..."
3,5,General Manager Scott Petello is a good egg!!!...
4,5,Drop what you're doing and drive here. After I...


Binary classification:

In [5]:
def is_good_review(num_stars):
    # This function categorizes reviews based on the number of stars.
    # It should return True if the review is positive (4 or 5 stars).
    # It should return False if the review is negative (1, 2, or 3 stars).

    # Replace 'None' with the appropriate condition for a positive review.
    if num_stars >= 4:  # YOUR CODE HERE
        return True # What should this return?
    else:
        return False

# Apply the function to the 'stars' column to create a new 'is_good_review' column.
# This column will have a Boolean value where True represents a 'good' review and False represents a 'bad' review.
yelp['is_good_review'] = yelp['stars'].apply(is_good_review)

# Display the first few rows to verify the changes.
yelp.head()

,stars,text,is_good_review
0,5,My wife took me here on my birthday for breakf...,True
1,5,I have no idea why some people give bad review...,True
2,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",True
3,5,General Manager Scott Petello is a good egg!!!...,True
4,5,Drop what you're doing and drive here. After I...,True


Training:

In [7]:
# Specify the input features (X) and output labels (y) from the 'yelp' dataframe
X_columns = "text" # Replace with the names of column to be used as input features
y_column = "is_good_review"  # Replace with the name of the column to be used as output labels


X_text = yelp[X_columns]
y = yelp[y_column]

def tokenize_and_embed(text_data):
    """
    Tokenizes the text data and converts it to word embeddings using SpaCy.
    Args:
        text_data (list): A list of text strings to be processed.
    Returns:
        list: A list of lists containing embeddings for each token in each document.
    """
    docs = list(text_to_nlp.pipe(text_data))
    embeddings = [[token.vector for token in doc] for doc in docs]
    return embeddings

def standardize_length(embeddings):
    """
    Ensures all embedding lists are the same length by padding shorter ones with zero vectors.
    Args:
        embeddings (list): A list of lists of embeddings.
    Returns:
        list: A list of lists with padded embeddings to ensure uniform length.
    """
    max_length = max(len(tokens) for tokens in embeddings)
    embedding_dim = len(embeddings[0][0]) if embeddings[0] else 0
    padded_embeddings = [[np.zeros(embedding_dim)] * (max_length - len(tokens)) + tokens for tokens in embeddings]
    return padded_embeddings

def convert_to_array(padded_embeddings):
    """
    Converts a list of padded embeddings into a numpy array.
    Args:
        padded_embeddings (list): A list of lists of padded embeddings.
    Returns:
        numpy.ndarray: A numpy array containing the embeddings suitable for machine learning input.
    """
    return np.array(padded_embeddings)

# X_text contains our text data to be processed
X_embeddings = tokenize_and_embed(X_text)  # Tokenize and get embeddings

X_padded = standardize_length(X_embeddings)  # Standardize lengths

X = convert_to_array(X_padded)  # Convert to numpy array suitable for model input

if X is not None: print(f"The shape of our dataset is now: {X.shape}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)

The shape of our dataset is now: (1000, 1085, 300)


Loading in the RNN model:

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

class RNNClassifier:
    def __init__(self, num_epochs=30, lstm_units=50, dropout_rate=0.7):
        self.num_epochs = num_epochs
        self.lstm_units = lstm_units
        self.dropout_rate = dropout_rate
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(LSTM(self.lstm_units, return_sequences=True))
        model.add(Dropout(self.dropout_rate))
        model.add(LSTM(self.lstm_units))
        model.add(Dropout(self.dropout_rate))
        model.add(Dense(1, activation='sigmoid'))
        optimizer = Adam(learning_rate=0.001)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return model

    def fit(self, X_train, y_train, X_val=None, y_val=None, **kwargs):

        """
        Comment is necessary due to how over complicated I made this. - joel

        Fits the model to the training data. Supports optional validation data.
        If validation data is provided, early stopping is used if not it's not! haha

        Args:
            X_train (array): Training data features.
            y_train (array): Training data labels.
            X_val (array, optional): Validation data features.
            y_val (array, optional): Validation data labels.
            **kwargs: Additional keyword arguments to pass to the model's fit method.

        Returns:
            A history object containing training history.
        """

        if X_train is None and y_train is None:
          print("Arguments are none. Retry with correct arguments.")
          return None

        callbacks = kwargs.pop('callbacks', [])

        if X_val is not None and y_val is not None:
            early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
            callbacks.append(early_stopping)
            return self.model.fit(X_train, y_train, epochs=self.num_epochs, validation_data=(X_val, y_val), callbacks=callbacks, batch_size=32, verbose=1, **kwargs)
        else:
            return self.model.fit(X_train, y_train, epochs=self.num_epochs, batch_size=32, verbose=1, callbacks=callbacks, **kwargs)

    def predict(self, *args, **kwargs):
        predictions = self.model.predict(*args, **kwargs)
        return (predictions > 0.5).astype(int)

    def predict_proba(self, *args, **kwargs):
        return self.model.predict(*args, **kwargs)

    def score(self, X, y):
        predictions = self.predict(X)
        return accuracy_score(y, predictions)

    def __getattr__(self, name):
        if name != 'predict' and name != 'predict_proba':
            return getattr(self.model, name)
        else:
            raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")

Training + testing:

In [9]:
rnn = RNNClassifier(num_epochs=3, lstm_units=50, dropout_rate=0.5)
rnn.fit(X_train, y_train)

y_pred = rnn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 115ms/step - accuracy: 0.6377 - loss: 0.6477
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.6911 - loss: 0.5938
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - accuracy: 0.7394 - loss: 0.5539
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step
0.76
